# 朴素贝叶斯 

## 训练
- 采用积极学习
- 训练过程实际上就是构建一个三级字典，存储对应的先验概率

In [1]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

In [33]:
from __future__ import division

In [4]:
train_data = pd.read_csv('../data/weather.csv')
train_data.head()

,outlook,temperature,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes


In [5]:
train_y = train_data.play
train_X = train_data.drop(['play'],axis=1,inplace=False)

In [7]:
train_X.shape
print type(train_y)

<class 'pandas.core.series.Series'>


In [8]:
resu = train_y.value_counts()
print type(resu)
print resu

<class 'pandas.core.series.Series'>
yes    9
no     5
Name: play, dtype: int64


In [11]:
dic = resu.to_dict()
print dic

{'yes': 9, 'no': 5}


In [14]:
train_y.unique()

array(['no', 'yes'], dtype=object)

In [22]:
train_X[train_y == 'yes']['outlook']

2     overcast
3        rainy
4        rainy
6     overcast
8        sunny
9        rainy
10       sunny
11    overcast
12    overcast
Name: outlook, dtype: object

In [29]:
# 假设输入数据带有标签，即pd.Dataframe，Series类型的数据
def fit(train_X,train_y,valid_X = None, valid_y = None):
    label_dic = train_y.value_counts().to_dict()
    x_columns = train_X.columns
    labels = train_y.unique()
    feature_dic = {}
    for label in labels:
        feature_dic[label] = {}
        for column in x_columns:
            relu = train_X[train_y == label][column].value_counts().to_dict()
            feature_dic[label][column] = relu
    return (label_dic,feature_dic)

In [30]:
label_dic , feature_dic = fit(train_X,train_y)

In [31]:
label_dic

{'no': 5, 'yes': 9}

In [32]:
feature_dic

{'no': {'humidity': {'high': 4, 'normal': 1},
  'outlook': {'rainy': 2, 'sunny': 3},
  'temperature': {'cool': 1, 'hot': 2, 'mild': 2},
  'windy': {False: 2, True: 3}},
 'yes': {'humidity': {'high': 3, 'normal': 6},
  'outlook': {'overcast': 4, 'rainy': 3, 'sunny': 2},
  'temperature': {'cool': 3, 'hot': 2, 'mild': 4},
  'windy': {False: 6, True: 3}}}

## 预测
- 预测过程实际上是根据训练过程中产生的先验概率来计算后验概率

### 使用拉普拉斯修正、对数化操作

In [40]:
for item in train_X.iloc[0]:
    print item

sunny
hot
high
False


In [53]:
for key,value in label_dic.items():
    print key,value
pre_y = pd.DataFrame(data=np.zeros(shape=(train_X.shape[0],2)),columns=label_dic.keys())

yes 9
no 5


In [ ]:
def predict(test_X):
    """
    type test_X:[[]]
    rtype []
    """
    m , n = train_X.shape
    d = len(train_y.unique())
    pre_y = pd.DataFrame(data=np.zeros(shape=(test_X.shape[0],d)),columns=proba_label.keys())
    for i in range(test_X.shape[0]):
        for key,value in label_dic.items():
            for item in test_X.columns:
                pre_y.iloc[i][key] += \
                np.log((feature_dic[key][item][test_X[item]]+1)/(label_dic[key]+len(feature_dic[key][item])))
            pre_y.iloc[i][key]+= np.log((value+1) / (m+d))
    pre_y = np.argmax(pre_y.values,axis=1)
    return pre_y